In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# base
import numpy as np
import pkg_resources
from sklearn.impute import SimpleImputer
from pandas_plink import read_plink

# viz
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# feems
from utils import prepare_graph_inputs
from spatial_graph import SpatialGraph
from viz import Viz
from CV import run_CV

# change matplotlib fonts
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.sans-serif"] = "Arial"

In [2]:
# data_path = pkg_resources.resource_filename("feems", "data/")
data_path = "./data"

# read the genotype data and mean impute missing data
(bim, fam, G) = read_plink("{}/wolvesadmix".format(data_path))
imp = SimpleImputer(missing_values=np.nan, strategy="mean")
genotypes = imp.fit_transform((np.array(G)).T)

print("n_samples={}, n_snps={}".format(genotypes.shape[0], genotypes.shape[1]))

# setup graph
coord = np.loadtxt("{}/wolvesadmix.coord".format(data_path))  # sample coordinates
outer = np.loadtxt("{}/wolvesadmix.outer".format(data_path))  # outer coordinates
grid_path = "{}/grid_100.shp".format(data_path)  # path to discrete global grid

# graph input files
outer, edges, grid, _ = prepare_graph_inputs(coord=coord, 
                                             ggrid=grid_path,
                                             translated=True, 
                                             buffer=0,
                                             outer=outer)

# spatial graph 
sp_graph = SpatialGraph(genotypes, coord, grid, edges)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 73.27it/s]


n_samples=111, n_snps=17729


In [7]:
# define grids
# reverse the order of lambdas and alphas for warmstart
lamb_grid = np.geomspace(1e-6, 1e2, 3)[::-1]
alpha_grid = np.geomspace(1e-5, 1e2, 3)[::-1]

# run cross-validation
cv_err = run_CV(sp_graph, lamb_grid, alpha_grid)

constant-w/variance fit, converged in 132 iterations, train_loss=2795677.6543430

 fold= 0
iteration lambda=3/3 alpha=3/3
 fold= 1
iteration lambda=3/3 alpha=3/3
 fold= 2
iteration lambda=3/3 alpha=3/3
 fold= 3
iteration lambda=3/3 alpha=3/3
 fold= 4
iteration lambda=3/3 alpha=3/3